### Retrieval-Augmented Generation with Wikipedia

In [1]:
import os
import openai
from gpt_helper import set_openai_api_key_from_txt,GPTchatClass,printmd
from wiki_helper import wiki_search
from util import printmd,extract_quoted_words
print ("openai version:[%s]"%(openai.__version__))

openai version:[0.27.8]


### Instantiate GPT Agent

In [2]:
set_openai_api_key_from_txt(key_path='../key/rilab_key.txt')
GPT = GPTchatClass(
    gpt_model='gpt-3.5-turbo', # 'gpt-3.5-turbo' / 'gpt-4'
    role_msg='Your are a helpful assistant summarizing infromation and answering user queries.')

OpenAI API Key Ready from [../key/rilab_key.txt].
Chat agent using [gpt-3.5-turbo] initialized with the follow role:[Your are a helpful assistant summarizing infromation and answering user queries.]


### Our RAG agent will use the following strategies
We assume that a user question is given (e.g., 'Who is the current president of South Korea?').
* Step 1. For the given question, our `GPT agent` will first generate a number of entities for searching Wikipedia.
* Step 2. Then, our `WikiBot` will provide (i.e., crawl) related information summarized with the `GPT agent` considering the user question.
* Step 3. Finally, the summarized texts and the original user question will be given to the `GPT agent` to answer. 

In [3]:
question = 'Who is the current president of South Korea?'
"""
question = '''
    I am an interactive humanoid robot agent. 
    I have following action capabilites:['idle','waving','greeting','raising hands','hugging','reading a book']
    I can detect following observations:['no people','a person appears','a person waves hands','a person leaves']
    I have a following personality:['Introverted and Childish']
    What is the best next action when I am in ['idle'] state and observes ['a person waves hands']?
'''
"""
print ("question: %s"%(question))

question: Who is the current president of South Korea?


### Step 1. Generate entities for wiki search

In [4]:
user_msg = \
    """
    Suppose you will use Wikipedia for retrieving information. 
    Could you recommend three query words wrapped with quotation marks considering the following question?
    """ + '"' + question + '"'

In [5]:
response_content = GPT.chat(
    user_msg=user_msg,PRINT_USER_MSG=True,PRINT_GPT_OUTPUT=True,
    RESET_CHAT=True,RETURN_RESPONSE=True)

[USER_MSG]



    Suppose you will use Wikipedia for retrieving information. 
    Could you recommend three query words wrapped with quotation marks considering the following question?
    "Who is the current president of South Korea?"

[GPT_OUTPUT]


Certainly! Here are three query words wrapped in quotation marks for the question "Who is the current president of South Korea?":

1. "Current president of South Korea"
2. "President of South Korea" 
3. "Current leader of South Korea"

In [6]:
# Print summarized sentence with a markdown format
printmd(response_content)

Certainly! Here are three query words wrapped in quotation marks for the question "Who is the current president of South Korea?":

1. "Current president of South Korea"
2. "President of South Korea" 
3. "Current leader of South Korea"

In [7]:
entities = extract_quoted_words(response_content)
if len(entities) > 3: entities = entities[-3:]
print (entities)

['Current president of South Korea', 'President of South Korea', 'Current leader of South Korea']


### Step 2. Query entities to `WikiBot`

In [8]:
paragraphs_return = []
for entity in entities:
    paragraphs_return += wiki_search(entity=entity,VERBOSE=True)

entity:[Current president of South Korea] mismatched. use [President of South Korea] instead.
 We have total [358] paragraphs.
 After filtering, we have [30] and [8] paragraphs returned (k:[5] and m:[3])
entity:[President of South Korea] matched.
 We have total [358] paragraphs.
 After filtering, we have [30] and [8] paragraphs returned (k:[5] and m:[3])
entity:[Current leader of South Korea] mismatched. use [List of current state leaders by date of assumption of office] instead.
 We have total [62] paragraphs.
 After filtering, we have [6] and [6] paragraphs returned (k:[5] and m:[3])


In [9]:
# Get the unique elements
paragraphs_unique = list(set(paragraphs_return))
print ("Number of paragraphs [%d] => unique ones [%d]"%
       (len(paragraphs_return),len(paragraphs_unique)))

Number of paragraphs [22] => unique ones [14]


In [10]:
# Now summarize each paragraph into a single sentence considering the question
summarized_sentences = []
for p_idx,p in enumerate(paragraphs_unique):
    user_msg = "You are given following question: "+question
    user_msg += "Could you summarize the following paragraph into one setence? \n "+p
    response_content = GPT.chat(
        user_msg=user_msg,PRINT_USER_MSG=False,PRINT_GPT_OUTPUT=False,
        RESET_CHAT=True,RETURN_RESPONSE=True)
    # Append summarized sentences
    summarized_sentences.append(response_content)
    # Print summarized sentence with a markdown format
    printmd(response_content)

Yoon Suk Yeol, a former prosecutor general and member of the conservative People Power Party, became the current president of South Korea on May 10, 2022, after winning the 2022 presidential election with a narrow 48.5% plurality.

The current president of South Korea is not mentioned in the provided paragraph.

The Provisional Government of the Republic of Korea, established in Shanghai in September 1919, played a key role in organizing the Korean people's resistance against the Japanese occupation, and its legitimacy has been recognized and succeeded by South Korea.

The paragraph describes the National Security Council and the Peaceful Unification Advisory Council in South Korea, with the latter serving as a government sounding board and a means to provide titles and opportunities to dignitaries and others.

The information provided does not contain any relevant information about the current president of South Korea.

...

The current president of South Korea is unable to seek re-election due to a law that has been in place since 1981, and the presidential term has been five years since 1988.

The current president of South Korea is unknown as it is not mentioned in the given paragraph.

The current president of South Korea is directly elected for a five-year term with no possibility of re-election, and if a vacancy occurs, a successor must be elected within sixty days.

The controversial Advisory Council of Elder Statesmen in South Korea, which was expanded and given cabinet rank, drew public suspicion that it was designed to benefit a certain individual, but those concerns were rendered moot when former President Chun withdrew to a Buddhist temple in self-imposed exile.

The paragraph explains that in parliamentary systems, the head of state and head of government are separate positions, whereas in presidential systems or dictatorships, one leader holds both positions, and in semi-presidential systems, both roles may be fulfilled by different individuals.

### Step 3. Answer the question using `summarized_sentences`

In [11]:
user_msg = " ".join(summarized_sentences)
user_msg += " Using the information above, could you answer the following question? "
user_msg += question

In [12]:
response_content = GPT.chat(
    user_msg=user_msg,PRINT_USER_MSG=True,PRINT_GPT_OUTPUT=True,
    RESET_CHAT=False,RETURN_RESPONSE=True)

[USER_MSG]


Yoon Suk Yeol, a former prosecutor general and member of the conservative People Power Party, became the current president of South Korea on May 10, 2022, after winning the 2022 presidential election with a narrow 48.5% plurality. The current president of South Korea is not mentioned in the provided paragraph. The Provisional Government of the Republic of Korea, established in Shanghai in September 1919, played a key role in organizing the Korean people's resistance against the Japanese occupation, and its legitimacy has been recognized and succeeded by South Korea. The paragraph describes the National Security Council and the Peaceful Unification Advisory Council in South Korea, with the latter serving as a government sounding board and a means to provide titles and opportunities to dignitaries and others. The information provided does not contain any relevant information about the current president of South Korea. The paragraph states that the list includes acting presidents, but only the current period of service is listed for leaders with non-consecutive terms. The president of South Korea is not allowed to dissolve the National Assembly, but they are permitted to take emergency measures that could amend or abolish existing laws, which must be approved by the National Assembly. The paragraph is discussing a list of current state leaders ranked by their continuous tenure in national leadership positions, including both head of state and head of government roles for countries where they are separate, and starting from when they took office rather than independence. The current president of South Korea is the head of state, head of government, and commander-in-chief of the Republic of Korea Armed Forces. The current president of South Korea is unable to seek re-election due to a law that has been in place since 1981, and the presidential term has been five years since 1988. The current president of South Korea is unknown as it is not mentioned in the given paragraph. The current president of South Korea is directly elected for a five-year term with no possibility of re-election, and if a vacancy occurs, a successor must be elected within sixty days. The controversial Advisory Council of Elder Statesmen in South Korea, which was expanded and given cabinet rank, drew public suspicion that it was designed to benefit a certain individual, but those concerns were rendered moot when former President Chun withdrew to a Buddhist temple in self-imposed exile. The paragraph explains that in parliamentary systems, the head of state and head of government are separate positions, whereas in presidential systems or dictatorships, one leader holds both positions, and in semi-presidential systems, both roles may be fulfilled by different individuals. Using the information above, could you answer the following question? Who is the current president of South Korea?

[GPT_OUTPUT]


The paragraph does not provide information about the current president of South Korea. Therefore, I cannot answer the question.

In [13]:
user_msg = "Could you explain about this little longer?"
response_content = GPT.chat(
    user_msg=user_msg,PRINT_USER_MSG=True,PRINT_GPT_OUTPUT=True,
    RESET_CHAT=False,RETURN_RESPONSE=True)

[USER_MSG]


Could you explain about this little longer?

[GPT_OUTPUT]


Certainly! The provided paragraph contains information about various aspects of South Korean politics and governance, but it does not mention the current president of South Korea. It discusses the establishment of the Provisional Government of the Republic of Korea, which played a significant role in organizing resistance against Japanese occupation. It also mentions the National Security Council and the Peaceful Unification Advisory Council in South Korea, which serve different roles within the government.

The paragraph goes on to mention that the president of South Korea is unable to dissolve the National Assembly but can take emergency measures that may amend or abolish existing laws with the approval of the National Assembly. It explains that the current president holds positions as the head of state, head of government, and commander-in-chief of the Republic of Korea Armed Forces.

However, despite discussing various elements of South Korean politics, the paragraph does not provide any information about the current president of South Korea. Therefore, the answer to the question of who the current president is cannot be found in the given paragraph.